In [2]:
pip install opencv-python

In [14]:
# Import

import tensorflow as tf
import os
import numpy as np
import cv2
import random
from PIL import Image
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from keras.utils import to_categorical
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0
from keras.src.metrics.confusion_metrics import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout,Activation
from tensorflow.keras.models import load_model

In [15]:
#Reading data

data_dir = './vegetables_fruits_data/'
train_folder = os.path.join(data_dir, 'train')
valid_folder = os.path.join(data_dir, 'validation')
test_folder = os.path.join(data_dir, 'test')
print(os.listdir(data_dir))
Cclasses = os.listdir(test_folder)
Cclasses

['test', 'train', 'validation']


['apple',
 'banana',
 'beetroot',
 'bell pepper',
 'cabbage',
 'capsicum',
 'carrot',
 'cauliflower',
 'chilli pepper',
 'corn',
 'cucumber',
 'eggplant',
 'garlic',
 'ginger',
 'grapes',
 'jalepeno',
 'kiwi',
 'lemon',
 'lettuce',
 'mango',
 'onion',
 'orange',
 'paprika',
 'pear',
 'peas',
 'pineapple',
 'pomegranate',
 'potato',
 'raddish',
 'soy beans',
 'spinach',
 'sweetcorn',
 'sweetpotato',
 'tomato',
 'turnip',
 'watermelon']

In [16]:
#deleting corrupted files

def filter_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)

            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                try:

                    with Image.open(file_path) as img:
                        img.verify()
                except OSError:

                    print(f"Removing: {file_path}")
                    os.remove(file_path)
            else:

                print(f"Removing: {file_path} - Unsupported file type")
                os.remove(file_path)

filter_images(test_folder)
filter_images(train_folder)
filter_images(valid_folder)

Implementing some calculation functions to view the number of files in folder so we can decide if we should use oversample or undersample

In [17]:
# Function to count the number of files in a folder
def count_files(folder_path):
    try:
        with os.scandir(folder_path) as entries:
            return sum(1 for entry in entries if entry.is_file() and not entry.name.startswith('.'))
    except OSError as e:
        print(f"Error counting files in {folder_path}: {e}")
        return 0

# Function to calculate the percentage of files in a folder
def calculate_percentage(folder_path, total_files):
    folder_files = count_files(folder_path)
    return (folder_files / total_files) * 100 if total_files > 0 else 0

# Iterate through test, train, and valid folders
for folder in [test_folder, train_folder, valid_folder]:
    total_files = sum([count_files(os.path.join(folder, class_name)) for class_name in Cclasses])

    print(f"Folder: {folder}")
    for class_name in Cclasses:
        class_folder = os.path.join(folder, class_name)
        class_count = count_files(class_folder)
        class_percentage = calculate_percentage(class_folder, total_files)
        print(f"  Class: {class_name} - Count: {class_count} - Percentage: {class_percentage:.2f}%")

    print(f"  Total Files: {total_files}")
    print()

Folder: ./vegetables_fruits_data/test
  Class: apple - Count: 10 - Percentage: 2.79%
  Class: banana - Count: 9 - Percentage: 2.51%
  Class: beetroot - Count: 10 - Percentage: 2.79%
  Class: bell pepper - Count: 10 - Percentage: 2.79%
  Class: cabbage - Count: 10 - Percentage: 2.79%
  Class: capsicum - Count: 10 - Percentage: 2.79%
  Class: carrot - Count: 10 - Percentage: 2.79%
  Class: cauliflower - Count: 10 - Percentage: 2.79%
  Class: chilli pepper - Count: 10 - Percentage: 2.79%
  Class: corn - Count: 10 - Percentage: 2.79%
  Class: cucumber - Count: 10 - Percentage: 2.79%
  Class: eggplant - Count: 10 - Percentage: 2.79%
  Class: garlic - Count: 10 - Percentage: 2.79%
  Class: ginger - Count: 10 - Percentage: 2.79%
  Class: grapes - Count: 10 - Percentage: 2.79%
  Class: jalepeno - Count: 10 - Percentage: 2.79%
  Class: kiwi - Count: 10 - Percentage: 2.79%
  Class: lemon - Count: 10 - Percentage: 2.79%
  Class: lettuce - Count: 10 - Percentage: 2.79%
  Class: mango - Count: 10 -